In [1]:
import pickle, gzip, numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import math


def plot_images(X):
    if X.ndim == 1:
        X = np.array([X])
    num_images = X.shape[0]
    num_rows = math.floor(math.sqrt(num_images))
    num_cols = math.ceil(num_images/num_rows)
    for i in range(num_images):
        reshaped_image = X[i,:].reshape(28,28)
        plt.subplot(num_rows, num_cols, i+1)
        plt.imshow(reshaped_image, cmap = cm.Greys_r)
        plt.axis('off')
    plt.show()


def pick_examples_of(X, Y, labels, total_count):
    bool_arr = None
    for label in labels:
        bool_arr_for_label = (Y == label)
        if bool_arr is None:
            bool_arr = bool_arr_for_label
        else:
            bool_arr |= bool_arr_for_label
    filtered_x = X[bool_arr]
    filtered_y = Y[bool_arr]
    return (filtered_x[:total_count], filtered_y[:total_count])


def extract_training_and_test_examples_with_labels(train_x, train_y, test_x, test_y, labels, training_count, test_count):
    filtered_train_x, filtered_train_y = pick_examples_of(train_x, train_y, labels, training_count)
    filtered_test_x, filtered_test_y = pick_examples_of(test_x, test_y, labels, test_count)
    return (filtered_train_x, filtered_train_y, filtered_test_x, filtered_test_y)

def write_pickle_data(data, file_name):
    f = gzip.open(file_name, 'wb')
    pickle.dump(data, f)
    f.close()

def read_pickle_data(file_name):
    f = gzip.open(file_name, 'rb')
    data = pickle.load(f, encoding='latin1')
    f.close()
    return data

def get_MNIST_data():
    """
    Reads mnist dataset from file

    Returns:
        train_x - 2D Numpy array (n, d) where each row is an image
        train_y - 1D Numpy array (n, ) where each row is a label
        test_x  - 2D Numpy array (n, d) where each row is an image
        test_y  - 1D Numpy array (n, ) where each row is a label

    """
    train_set, valid_set, test_set = read_pickle_data('Datasets/mnist.pkl.gz')
    train_x, train_y = train_set
    valid_x, valid_y = valid_set
    train_x = np.vstack((train_x, valid_x))
    train_y = np.append(train_y, valid_y)
    test_x, test_y = test_set
    return (train_x, train_y, test_x, test_y)

def load_train_and_test_pickle(file_name):
    train_x, train_y, test_x, test_y = read_pickle_data(file_name)
    return train_x, train_y, test_x, test_y

# returns the feature set in a numpy ndarray
def load_CSV(filename):
    stuff = np.asarray(np.loadtxt(open(filename, 'rb'), delimiter=','))
    return stuff


In [2]:
import numpy as np
import math

"""
 ==================================
 Problem 3: Neural Network Basics
 ==================================
    Generates a neural network with the following architecture:
        Fully connected neural network.
        Input vector takes in two features.
        One hidden layer with three neurons whose activation function is ReLU.
        One output neuron whose activation function is the identity function.
"""


def rectified_linear_unit(x):
    """ Returns the ReLU of x, or the maximum between 0 and x."""
    return np.maximum(0, x)

def rectified_linear_unit_derivative(x):
    """Returns the derivative of ReLU."""
    if x > 0:
        return 1
    else:
        return 0

def output_layer_activation(x):
    """ Linear function, returns input as is. """
    return x

def output_layer_activation_derivative(x):
    """ Returns the derivative of a linear function: 1. """
    return 1

class NeuralNetwork():
    """
        Contains the following functions:
            -train: tunes parameters of the neural network based on error obtained from forward propagation.
            -predict: predicts the label of a feature vector based on the class's parameters.
            -train_neural_network: trains a neural network over all the data points for the specified number of epochs during initialization of the class.
            -test_neural_network: uses the parameters specified at the time in order to test that the neural network classifies the points given in testing_points within a margin of error.
    """

    def __init__(self):

        # DO NOT CHANGE PARAMETERS (Initialized to floats instead of ints)
        self.input_to_hidden_weights = np.matrix('1. 1.; 1. 1.; 1. 1.')
        self.hidden_to_output_weights = np.matrix('1. 1. 1.')
        self.biases = np.matrix('0.; 0.; 0.')
        self.learning_rate = .001
        self.epochs_to_train = 10
        self.training_points = [((2,1), 10), ((3,3), 21), ((4,5), 32), ((6, 6), 42)]
        self.testing_points = [(1,1), (2,2), (3,3), (5,5), (10,10)]

    def train(self, x1, x2, y):

        ### Forward propagation ###
        input_values = np.matrix([[x1], [x2]])  # 2 by 1

        # Calculate the input and activation of the hidden layer
        hidden_layer_weighted_input = (
                self.input_to_hidden_weights.dot(input_values) + self.biases
        )  # (3 by 1 matrix)
        ReLU_vec = np.vectorize(rectified_linear_unit)  # Vectorize ReLU function
        hidden_layer_activation = ReLU_vec(
            hidden_layer_weighted_input
        )  # (3 by 1 matrix)

        output = self.hidden_to_output_weights.dot(hidden_layer_activation)
        activated_output = output_layer_activation(output)

        ### Backpropagation ###

        # Compute gradients
        output_layer_error = -(y - activated_output)  # dC/df(u1)

        output_derivative_vec = np.vectorize(
            output_layer_activation_derivative
        )  # Vectorize derivative of output activation
        hidden_layer_error = (
                np.multiply(
                    output_derivative_vec(activated_output),
                    self.hidden_to_output_weights.transpose(),
                )
                * output_layer_error
        )  # (3 by 1 matrix)

        ReLU_derivative_vec = np.vectorize(
            rectified_linear_unit_derivative
        )  # Vectorize ReLU derivative
        bias_gradients = np.multiply(
            hidden_layer_error, ReLU_derivative_vec(hidden_layer_weighted_input)
        )  # dC/db

        hidden_to_output_weight_gradients = np.multiply(
            hidden_layer_activation, output_layer_error
        ).transpose()  # dC/dV
        input_to_hidden_weight_gradients = bias_gradients.dot(
            input_values.transpose()
        )  # dC/dW

        # Use gradients to adjust weights and biases using gradient descent
        self.biases = self.biases - self.learning_rate * bias_gradients
        self.input_to_hidden_weights = (
                self.input_to_hidden_weights
                - self.learning_rate * input_to_hidden_weight_gradients
        )
        self.hidden_to_output_weights = (
                self.hidden_to_output_weights
                - self.learning_rate * hidden_to_output_weight_gradients
        )

    def predict(self, x1, x2):

        input_values = np.matrix([[x1], [x2]])

        # Compute output for a single input(should be same as the forward propagation in training)
        hidden_layer_weighted_input = (
                self.input_to_hidden_weights.dot(input_values) + self.biases
        )
        ReLU_vec = np.vectorize(rectified_linear_unit)
        hidden_layer_activation = ReLU_vec(hidden_layer_weighted_input)

        output = self.hidden_to_output_weights.dot(hidden_layer_activation)
        activated_output = output_layer_activation(output)

        return activated_output.item()

    # Run this to train your neural network once you complete the train method
    def train_neural_network(self):

        for epoch in range(self.epochs_to_train):
            for x,y in self.training_points:
                self.train(x[0], x[1], y)

    # Run this to test your neural network implementation for correctness after it is trained
    def test_neural_network(self):

        for point in self.testing_points:
            print("Point,", point, "Prediction,", self.predict(point[0], point[1]))
            if abs(self.predict(point[0], point[1]) - 7*point[0]) < 0.1:
                print("Test Passed")
            else:
                print("Point ", point[0], point[1], " failed to be predicted correctly.")
                return

x = NeuralNetwork()

x.train_neural_network()

In [3]:
x.test_neural_network()

Point, (1, 1) Prediction, 7.038453196038151
Test Passed
Point, (2, 2) Prediction, 14.042814826755537
Test Passed
Point, (3, 3) Prediction, 21.047176457472922
Test Passed
Point, (5, 5) Prediction, 35.055899718907696
Test Passed
Point, (10, 10) Prediction, 70.07770787249461
Test Passed


In [5]:
"""Training utilities."""

from tqdm import tqdm
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn


class Flatten(nn.Module):
    """A custom layer that views an input as 1D."""

    def forward(self, input):
        return input.view(input.size(0), -1)

# Helpers
def batchify_data(x_data, y_data, batch_size):
    """Takes a set of data points and labels and groups them into batches."""
    # Only take batch_size chunks (i.e. drop the remainder)
    N = int(len(x_data) / batch_size) * batch_size
    batches = []
    for i in range(0, N, batch_size):
        batches.append({
            'x': torch.tensor(x_data[i:i+batch_size], dtype=torch.float32),
            'y': torch.tensor(y_data[i:i+batch_size], dtype=torch.long
                              )})
    return batches

def compute_accuracy(predictions, y):
    """Computes the accuracy of predictions against the gold labels, y."""
    return np.mean(np.equal(predictions.numpy(), y.numpy()))


# Training Procedure
def train_model(train_data, dev_data, model, lr=0.01, momentum=0.9, nesterov=False, n_epochs=30):
    """Train a model for N epochs given data and hyper-params."""
    # We optimize with SGD
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, nesterov=nesterov)

    for epoch in range(1, 11):
        print("-------------\nEpoch {}:\n".format(epoch))


        # Run **training***
        loss, acc = run_epoch(train_data, model.train(), optimizer)
        print('Train loss: {:.6f} | Train accuracy: {:.6f}'.format(loss, acc))

        # Run **validation**
        val_loss, val_acc = run_epoch(dev_data, model.eval(), optimizer)
        print('Val loss:   {:.6f} | Val accuracy:   {:.6f}'.format(val_loss, val_acc))
        # Save model
        torch.save(model, 'mnist_model_fully_connected.pt')
    return val_acc

def run_epoch(data, model, optimizer):
    """Train model for one pass of train data, and return loss, acccuracy"""
    # Gather losses
    losses = []
    batch_accuracies = []

    # If model is in train mode, use optimizer.
    is_training = model.training

    # Iterate through batches
    for batch in tqdm(data):
        # Grab x and y
        x, y = batch['x'], batch['y']

        # Get output predictions
        out = model(x)

        # Predict and store accuracy
        predictions = torch.argmax(out, dim=1)
        batch_accuracies.append(compute_accuracy(predictions, y))

        # Compute loss
        loss = F.cross_entropy(out, y)
        losses.append(loss.data.item())

        # If training, do an update.
        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Calculate epoch level scores
    avg_loss = np.mean(losses)
    avg_accuracy = np.mean(batch_accuracies)
    return avg_loss, avg_accuracy


#  Training and Testing Accuracy Over Time 

In [7]:
import _pickle as cPickle, gzip
import numpy as np
from tqdm import tqdm
import torch
import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn as nn
import sys
sys.path.append("..")
import warnings
# Suppress warnings:
def warn(*args, **kwargs):
    pass

warnings.warn = warn

def main():
    # Load the dataset
    num_classes = 10
    X_train, y_train, X_test, y_test = get_MNIST_data()

    # Split into train and dev
    dev_split_index = int(9 * len(X_train) / 10)
    X_dev = X_train[dev_split_index:]
    y_dev = y_train[dev_split_index:]
    X_train = X_train[:dev_split_index]
    y_train = y_train[:dev_split_index]

    permutation = np.array([i for i in range(len(X_train))])
    np.random.shuffle(permutation)
    X_train = [X_train[i] for i in permutation]
    y_train = [y_train[i] for i in permutation]

    # Split dataset into batches
    batch_size = 32
    train_batches = batchify_data(X_train, y_train, batch_size)
    dev_batches = batchify_data(X_dev, y_dev, batch_size)
    test_batches = batchify_data(X_test, y_test, batch_size)

    #################################
    ## Model specification TODO
    model = nn.Sequential(
        nn.Linear(784, 10),
        nn.ReLU(),
        nn.Linear(10, 10),
    )
    lr=0.1
    momentum=0
    ##################################

    train_model(train_batches, dev_batches, model, lr=lr, momentum=momentum)

    ## Evaluate the model on test data
    loss, accuracy = run_epoch(test_batches, model.eval(), None)

    print ("Loss on test set:"  + str(loss) + " Accuracy on test set: " + str(accuracy))


if __name__ == '__main__':
    # Specify seed for deterministic behavior, then shuffle. Do not change seed for official submissions to edx
    np.random.seed(12321)  # for reproducibility
    torch.manual_seed(12321)  # for reproducibility
    main()

-------------
Epoch 1:


100%|██████████| 1687/1687 [00:03<00:00, 492.02it/s]


Train loss: 0.445684 | Train accuracy: 0.870758


100%|██████████| 187/187 [00:00<00:00, 2047.95it/s]


Val loss:   0.260342 | Val accuracy:   0.923630
-------------
Epoch 2:


...

Train loss: 0.223966 | Train accuracy: 0.935147


100%|██████████| 187/187 [00:00<00:00, 1590.55it/s]


Val loss:   0.228388 | Val accuracy:   0.932487


100%|██████████| 312/312 [00:00<00:00, 1846.79it/s]


Loss on test set:0.26722689023993623 Accuracy on test set: 0.9204727564102564


# Improving Accuracy 

In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

def main(modification):
    # Set seeds
    np.random.seed(12321)
    torch.manual_seed(12321)

    # Load the dataset
    num_classes = 10
    X_train, y_train, X_test, y_test = get_MNIST_data()

    # Split into train and dev
    dev_split_index = int(9 * len(X_train) / 10)
    X_dev = X_train[dev_split_index:]
    y_dev = y_train[dev_split_index:]
    X_train = X_train[:dev_split_index]
    y_train = y_train[:dev_split_index]

    permutation = np.array([i for i in range(len(X_train))])
    np.random.shuffle(permutation)
    X_train = [X_train[i] for i in permutation]
    y_train = [y_train[i] for i in permutation]

    # Set default values
    batch_size = 32
    lr = 0.1
    momentum = 0
    activation_function = nn.ReLU()

    # Make necessary changes based on the modification
    if modification == "batch size 64":
        batch_size = 64
    elif modification == "learning rate 0.01":
        lr = 0.01
    elif modification == "momentum 0.9":
        momentum = 0.9
    elif modification == "LeakyReLU activation":
        activation_function = nn.LeakyReLU()

    # Split dataset into batches
    train_batches = batchify_data(X_train, y_train, batch_size)
    dev_batches = batchify_data(X_dev, y_dev, batch_size)
    test_batches = batchify_data(X_test, y_test, batch_size)

    # Model
    model = nn.Sequential(
        nn.Linear(784, 10),
        activation_function,
        nn.Linear(10, 10),
    )

    # Train
    val_accuracy = train_model(train_batches, dev_batches, model, lr=lr, momentum=momentum)

    return val_accuracy


if __name__ == '__main__':
    best_accuracy = 0
    best_modification = "baseline"

    modifications = ["baseline", "batch size 64", "learning rate 0.01", "momentum 0.9", "LeakyReLU activation"]

    for modification in modifications:
        val_accuracy = main(modification)
        print(modification, "achieved validation accuracy of:", val_accuracy)

        # Update best accuracy if this run is better
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_modification = modification

    print("Best modification is:", best_modification, "with validation accuracy:", best_accuracy)

-------------
Epoch 1:


100%|██████████| 1687/1687 [00:02<00:00, 596.04it/s]


Train loss: 0.445684 | Train accuracy: 0.870758


100%|██████████| 187/187 [00:00<00:00, 2160.79it/s]


Val loss:   0.260342 | Val accuracy:   0.923630
-------------
Epoch 2:


...

Val loss:   0.228252 | Val accuracy:   0.932487
-------------
Epoch 10:


100%|██████████| 1687/1687 [00:03<00:00, 524.54it/s]


Train loss: 0.224515 | Train accuracy: 0.935147


100%|██████████| 187/187 [00:00<00:00, 2065.41it/s]


Val loss:   0.227615 | Val accuracy:   0.931985
LeakyReLU activation achieved validation accuracy of: 0.9319852941176471
Best modification is: batch size 64 with validation accuracy: 0.9400201612903226


In [9]:
def main(modification):
    # Set seeds
    np.random.seed(12321)
    torch.manual_seed(12321)

    # Load the dataset
    num_classes = 10
    X_train, y_train, X_test, y_test = get_MNIST_data()

    # Split into train and dev
    dev_split_index = int(9 * len(X_train) / 10)
    X_dev = X_train[dev_split_index:]
    y_dev = y_train[dev_split_index:]
    X_train = X_train[:dev_split_index]
    y_train = y_train[:dev_split_index]

    permutation = np.array([i for i in range(len(X_train))])
    np.random.shuffle(permutation)
    X_train = [X_train[i] for i in permutation]
    y_train = [y_train[i] for i in permutation]

    # Set default values
    batch_size = 32
    lr = 0.1
    momentum = 0
    activation_function = nn.ReLU()

    # Make necessary changes based on the modification
    if modification == "batch size 64":
        batch_size = 64
    elif modification == "learning rate 0.01":
        lr = 0.01
    elif modification == "momentum 0.9":
        momentum = 0.9
    elif modification == "LeakyReLU activation":
        activation_function = nn.LeakyReLU()

    # Split dataset into batches
    train_batches = batchify_data(X_train, y_train, batch_size)
    dev_batches = batchify_data(X_dev, y_dev, batch_size)
    test_batches = batchify_data(X_test, y_test, batch_size)

    # Model
    model = nn.Sequential(
        nn.Linear(784, 10),
        activation_function,
        nn.Linear(10, 10),
    )

    # Train
    val_accuracy = train_model(train_batches, dev_batches, model, lr=lr, momentum=momentum)
    _, test_accuracy = run_epoch(test_batches, model.eval(), None)

    return val_accuracy, test_accuracy


if __name__ == '__main__':
    best_val_accuracy = 0
    best_test_accuracy = 0
    best_modification_val = "baseline"
    best_modification_test = "baseline"

    modifications = ["baseline", "batch size 64", "learning rate 0.01", "momentum 0.9", "LeakyReLU activation"]

    for modification in modifications:
        val_accuracy, test_accuracy = main(modification)
        print(modification, "achieved validation accuracy of:", val_accuracy, "and test accuracy of:", test_accuracy)

        # Update best accuracies if this run is better
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_modification_val = modification
        if test_accuracy > best_test_accuracy:
            best_test_accuracy = test_accuracy
            best_modification_test = modification

    print("Best modification based on validation accuracy is:", best_modification_val, "with accuracy:", best_val_accuracy)
    print("Best modification based on test accuracy is:", best_modification_test, "with accuracy:", best_test_accuracy)

-------------
Epoch 1:


100%|██████████| 1687/1687 [00:02<00:00, 595.73it/s]


Train loss: 0.445684 | Train accuracy: 0.870758


100%|██████████| 187/187 [00:00<00:00, 2688.38it/s]


Val loss:   0.260342 | Val accuracy:   0.923630
-------------
Epoch 2:


...

Train loss: 0.224515 | Train accuracy: 0.935147


100%|██████████| 187/187 [00:00<00:00, 2322.19it/s]


Val loss:   0.227615 | Val accuracy:   0.931985


100%|██████████| 312/312 [00:00<00:00, 2276.32it/s]

LeakyReLU activation achieved validation accuracy of: 0.9319852941176471 and test accuracy of: 0.9207732371794872
Best modification based on validation accuracy is: batch size 64 with accuracy: 0.9400201612903226
Best modification based on test accuracy is: batch size 64 with accuracy: 0.9314903846153846


# Improving Accuracy - Hidden 128

In [10]:
def main(modification):
    # Load the dataset
    num_classes = 10
    X_train, y_train, X_test, y_test = get_MNIST_data()

    # Split into train and dev
    dev_split_index = int(9 * len(X_train) / 10)
    X_dev = X_train[dev_split_index:]
    y_dev = y_train[dev_split_index:]
    X_train = X_train[:dev_split_index]
    y_train = y_train[:dev_split_index]

    permutation = np.array([i for i in range(len(X_train))])
    np.random.shuffle(permutation)
    X_train = [X_train[i] for i in permutation]
    y_train = [y_train[i] for i in permutation]

    # Modify the model's architecture
    model = nn.Sequential(
        nn.Linear(784, 128),
        nn.ReLU() if modification != "LeakyReLU activation" else nn.LeakyReLU(),
        nn.Linear(128, 10),
    )

    # Hyperparameters
    batch_size = 32
    lr = 0.1
    momentum = 0

    if modification == "batch size 64":
        batch_size = 64
    elif modification == "learning rate 0.01":
        lr = 0.01
    elif modification == "momentum 0.9":
        momentum = 0.9

    # Split dataset into batches
    train_batches = batchify_data(X_train, y_train, batch_size)
    dev_batches = batchify_data(X_dev, y_dev, batch_size)
    test_batches = batchify_data(X_test, y_test, batch_size)

    val_acc = train_model(train_batches, dev_batches, model, lr=lr, momentum=momentum)

    return val_acc


if __name__ == '__main__':
    np.random.seed(12321)  # for reproducibility
    torch.manual_seed(12321)  # for reproducibility

    modifications = [
        "baseline (no modifications)",
        "batch size 64",
        "learning rate 0.01",
        "momentum 0.9",
        "LeakyReLU activation"
    ]

    best_modification = None
    best_val_acc = 0

    for modification in modifications:
        val_acc = main(modification)
        print(f"Validation accuracy with {modification}: {val_acc}")
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_modification = modification

    print(f"\nBest modification is: {best_modification} with validation accuracy: {best_val_acc}")

-------------
Epoch 1:


100%|██████████| 1687/1687 [00:03<00:00, 459.61it/s]


Train loss: 0.366369 | Train accuracy: 0.897081


100%|██████████| 187/187 [00:00<00:00, 1649.68it/s]


Val loss:   0.178036 | Val accuracy:   0.947861
-------------
Epoch 2:


...

Val loss:   0.071080 | Val accuracy:   0.979612
-------------
Epoch 10:


100%|██████████| 1687/1687 [00:04<00:00, 390.57it/s]


Train loss: 0.036325 | Train accuracy: 0.990738


100%|██████████| 187/187 [00:00<00:00, 1306.89it/s]


Val loss:   0.070506 | Val accuracy:   0.979779
Validation accuracy with LeakyReLU activation: 0.9797794117647058

Best modification is: LeakyReLU activation with validation accuracy: 0.9797794117647058


# Convolutional Neural Networks

In [11]:
import _pickle as c_pickle, gzip
import numpy as np
from tqdm import tqdm
import torch
import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn as nn
import sys
sys.path.append("..")


def main():
    # Load the dataset
    num_classes = 10
    X_train, y_train, X_test, y_test = get_MNIST_data()

    # We need to rehape the data back into a 1x28x28 image
    X_train = np.reshape(X_train, (X_train.shape[0], 1, 28, 28))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, 28, 28))

    # Split into train and dev
    dev_split_index = int(9 * len(X_train) / 10)
    X_dev = X_train[dev_split_index:]
    y_dev = y_train[dev_split_index:]
    X_train = X_train[:dev_split_index]
    y_train = y_train[:dev_split_index]

    permutation = np.array([i for i in range(len(X_train))])
    np.random.shuffle(permutation)
    X_train = [X_train[i] for i in permutation]
    y_train = [y_train[i] for i in permutation]

    # Split dataset into batches
    batch_size = 32
    train_batches = batchify_data(X_train, y_train, batch_size)
    dev_batches = batchify_data(X_dev, y_dev, batch_size)
    test_batches = batchify_data(X_test, y_test, batch_size)

    #################################
    ## Model specification TODO
    model = nn.Sequential(
        nn.Conv2d(1, 32, (3, 3)),
        nn.ReLU(),
        nn.MaxPool2d((2, 2)),
        nn.Conv2d(32, 64, (3,3)),
        nn.ReLU(),
        nn.MaxPool2d((2, 2)),
        Flatten(),
        nn.Linear(1600,128),
        nn.Dropout(p = 0.5),
        nn.Linear(128,10)
    )
    ##################################

    train_model(train_batches, dev_batches, model, nesterov=True)

    ## Evaluate the model on test data
    loss, accuracy = run_epoch(test_batches, model.eval(), None)

    print ("Loss on test set:"  + str(loss) + " Accuracy on test set: " + str(accuracy))


if __name__ == '__main__':
    # Specify seed for deterministic behavior, then shuffle. Do not change seed for official submissions to edx
    np.random.seed(12321)  # for reproducibility
    torch.manual_seed(12321)
    main()


-------------
Epoch 1:


100%|██████████| 1687/1687 [00:38<00:00, 44.11it/s]


Train loss: 0.244168 | Train accuracy: 0.923292


100%|██████████| 187/187 [00:01<00:00, 119.05it/s]


Val loss:   0.059497 | Val accuracy:   0.982955
-------------
Epoch 2:


...

Train loss: 0.020146 | Train accuracy: 0.993646


100%|██████████| 187/187 [00:01<00:00, 111.14it/s]


Val loss:   0.035877 | Val accuracy:   0.990976


100%|██████████| 312/312 [00:02<00:00, 105.63it/s]

Loss on test set:0.030044354157752924 Accuracy on test set: 0.9895833333333334


# Overlapping, multi-digit MNIST

In [12]:
"""Training utilities."""

from tqdm import tqdm
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn


class Flatten(nn.Module):
    """A custom layer that views an input as 1D."""

    def forward(self, input):
        return input.view(input.size(0), -1)


def batchify_data(x_data, y_data, batch_size):
    """Takes a set of data points and labels and groups them into batches."""
    # Only take batch_size chunks (i.e. drop the remainder)
    N = int(len(x_data) / batch_size) * batch_size
    batches = []
    for i in range(0, N, batch_size):
        batches.append({
            'x': torch.tensor(x_data[i:i + batch_size],
                              dtype=torch.float32),
            'y': torch.tensor([y_data[0][i:i + batch_size],
                               y_data[1][i:i + batch_size]],
                              dtype=torch.int64)
        })
    return batches


def compute_accuracy(predictions, y):
    """Computes the accuracy of predictions against the gold labels, y."""
    return np.mean(np.equal(predictions.numpy(), y.numpy()))


def train_model(train_data, dev_data, model, lr=0.01, momentum=0.9, nesterov=False, n_epochs=30):
    """Train a model for N epochs given data and hyper-params."""
    # We optimize with SGD
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, nesterov=nesterov)

    for epoch in range(1, n_epochs + 1):
        print("-------------\nEpoch {}:\n".format(epoch))

        # Run **training***
        loss, acc = run_epoch(train_data, model.train(), optimizer)
        print('Train | loss1: {:.6f}  accuracy1: {:.6f} | loss2: {:.6f}  accuracy2: {:.6f}'.format(loss[0], acc[0], loss[1], acc[1]))

        # Run **validation**
        val_loss, val_acc = run_epoch(dev_data, model.eval(), optimizer)
        print('Valid | loss1: {:.6f}  accuracy1: {:.6f} | loss2: {:.6f}  accuracy2: {:.6f}'.format(val_loss[0], val_acc[0], val_loss[1], val_acc[1]))

        # Save model
        torch.save(model, 'mnist_model_fully_connected.pt')


def run_epoch(data, model, optimizer):
    """Train model for one pass of train data, and return loss, acccuracy"""
    # Gather losses
    losses_first_label = []
    losses_second_label = []
    batch_accuracies_first = []
    batch_accuracies_second = []

    # If model is in train mode, use optimizer.
    is_training = model.training

    # Iterate through batches
    for batch in tqdm(data):
        # Grab x and y
        x, y = batch['x'], batch['y']

        # Get output predictions for both the upper and lower numbers
        out1, out2 = model(x)

        # Predict and store accuracy
        predictions_first_label = torch.argmax(out1, dim=1)
        predictions_second_label = torch.argmax(out2, dim=1)
        batch_accuracies_first.append(compute_accuracy(predictions_first_label, y[0]))
        batch_accuracies_second.append(compute_accuracy(predictions_second_label, y[1]))

        # Compute both losses
        loss1 = F.cross_entropy(out1, y[0])
        loss2 = F.cross_entropy(out2, y[1])
        losses_first_label.append(loss1.data.item())
        losses_second_label.append(loss2.data.item())

        # If training, do an update.
        if is_training:
            optimizer.zero_grad()
            joint_loss = 0.5 * (loss1 + loss2)
            joint_loss.backward()
            optimizer.step()

    # Calculate epoch level scores
    avg_loss = np.mean(losses_first_label), np.mean(losses_second_label)
    avg_accuracy = np.mean(batch_accuracies_first), np.mean(batch_accuracies_second)
    return avg_loss, avg_accuracy


In [13]:
import gzip, _pickle, numpy as np
num_classes = 10
img_rows, img_cols = 42, 28

def get_data(path_to_data_dir, use_mini_dataset):
    if use_mini_dataset:
        exten = '_mini'
    else:
        exten = ''
    f = gzip.open(path_to_data_dir + 'train_multi_digit' + exten + '.pkl.gz', 'rb')
    X_train = _pickle.load(f, encoding='latin1')
    f.close()
    X_train =  np.reshape(X_train, (len(X_train), 1, img_rows, img_cols))
    f = gzip.open(path_to_data_dir + 'test_multi_digit' + exten +'.pkl.gz', 'rb')
    X_test = _pickle.load(f, encoding='latin1')
    f.close()
    X_test =  np.reshape(X_test, (len(X_test),1, img_rows, img_cols))
    f = gzip.open(path_to_data_dir + 'train_labels' + exten +'.txt.gz', 'rb')
    y_train = np.loadtxt(f)
    f.close()
    f = gzip.open(path_to_data_dir +'test_labels' + exten + '.txt.gz', 'rb')
    y_test = np.loadtxt(f)
    f.close()
    return X_train, y_train, X_test, y_test


In [14]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
path_to_data_dir = 'Datasets/'
use_mini_dataset = True

batch_size = 64
nb_classes = 10
nb_epoch = 30
num_classes = 10
img_rows, img_cols = 42, 28 # input image dimensions

class MLP(nn.Module):

    def __init__(self, input_dimension):
        super(MLP, self).__init__()
        self.flatten = Flatten()
        self.linear1 = nn.Linear(input_dimension, 64)
        self.linear2 = nn.Linear(64, 20)

    def forward(self, x):
        xf = self.flatten(x)

        xl1 = self.linear1(xf)
        xl2 = self.linear2(xl1)
        out_first_digit = xl2[:,:10]
        out_second_digit = xl2[:,10:]

        return out_first_digit, out_second_digit

def main():
    X_train, y_train, X_test, y_test = get_data(path_to_data_dir, use_mini_dataset)

    # Split into train and dev
    dev_split_index = int(9 * len(X_train) / 10)
    X_dev = X_train[dev_split_index:]
    y_dev = [y_train[0][dev_split_index:], y_train[1][dev_split_index:]]
    X_train = X_train[:dev_split_index]
    y_train = [y_train[0][:dev_split_index], y_train[1][:dev_split_index]]

    permutation = np.array([i for i in range(len(X_train))])
    np.random.shuffle(permutation)
    X_train = [X_train[i] for i in permutation]
    y_train = [[y_train[0][i] for i in permutation], [y_train[1][i] for i in permutation]]

    # Split dataset into batches
    train_batches = batchify_data(X_train, y_train, batch_size)
    dev_batches = batchify_data(X_dev, y_dev, batch_size)
    test_batches = batchify_data(X_test, y_test, batch_size)

    # Load model
    input_dimension = img_rows * img_cols
    model = MLP(input_dimension) # TODO add proper layers to MLP class above

    # Train
    train_model(train_batches, dev_batches, model)

    ## Evaluate the model on test data
    loss, acc = run_epoch(test_batches, model.eval(), None)
    print('Test loss1: {:.6f}  accuracy1: {:.6f}  loss2: {:.6f}   accuracy2: {:.6f}'.format(loss[0], acc[0], loss[1], acc[1]))

if __name__ == '__main__':
    # Specify seed for deterministic behavior, then shuffle. Do not change seed for official submissions to edx
    np.random.seed(12321)  # for reproducibility
    torch.manual_seed(12321)  # for reproducibility
    main()


C:\Users\ASUS\AppData\Local\Temp\ipykernel_11420\956033108.py:26: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  'y': torch.tensor([y_data[0][i:i + batch_size],


-------------
Epoch 1:


100%|██████████| 562/562 [00:01<00:00, 322.60it/s]


Train | loss1: 0.776068  accuracy1: 0.792538 | loss2: 0.798555  accuracy2: 0.777441


100%|██████████| 62/62 [00:00<00:00, 1077.76it/s]


...

Train | loss1: 0.260008  accuracy1: 0.924766 | loss2: 0.285256  accuracy2: 0.916315


100%|██████████| 62/62 [00:00<00:00, 985.30it/s]


Valid | loss1: 0.368430  accuracy1: 0.892389 | loss2: 0.353787  accuracy2: 0.898185


100%|██████████| 62/62 [00:00<00:00, 762.45it/s]


Test loss1: 0.400817  accuracy1: 0.892389  loss2: 0.374367   accuracy2: 0.893901


In [15]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
path_to_data_dir = 'Datasets/'
use_mini_dataset = True

batch_size = 64
nb_classes = 10
nb_epoch = 30
num_classes = 10
img_rows, img_cols = 42, 28 # input image dimensions



class CNN(nn.Module):

    def __init__(self, input_dimension):
        super(CNN, self).__init__()
        self.conv2d_1 = nn.Conv2d(1, 32, (3, 3))
        self.relu = nn.ReLU()
        self.maxpool2d = nn.MaxPool2d((2,2))
        self.conv2d_2 = nn.Conv2d(32, 64, (3, 3))
        self.flatten = Flatten()
        self.linear1 = nn.Linear(2880, 64)
        self.dropout = nn.Dropout(p = 0.5)
        self.linear2 = nn.Linear(64, 20)

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.relu(x)
        x = self.maxpool2d(x)
        x = self.conv2d_2(x)
        x = self.relu(x)
        x = self.maxpool2d(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.dropout(x)
        x = self.linear2(x)

        out_first_digit = x[:,:10]
        out_second_digit = x[:,10:]

        return out_first_digit, out_second_digit

def main():
    X_train, y_train, X_test, y_test = get_data(path_to_data_dir, use_mini_dataset)

    # Split into train and dev
    dev_split_index = int(9 * len(X_train) / 10)
    X_dev = X_train[dev_split_index:]
    y_dev = [y_train[0][dev_split_index:], y_train[1][dev_split_index:]]
    X_train = X_train[:dev_split_index]
    y_train = [y_train[0][:dev_split_index], y_train[1][:dev_split_index]]

    permutation = np.array([i for i in range(len(X_train))])
    np.random.shuffle(permutation)
    X_train = [X_train[i] for i in permutation]
    y_train = [[y_train[0][i] for i in permutation], [y_train[1][i] for i in permutation]]

    # Split dataset into batches
    train_batches = batchify_data(X_train, y_train, batch_size)
    dev_batches = batchify_data(X_dev, y_dev, batch_size)
    test_batches = batchify_data(X_test, y_test, batch_size)

    # Load model
    input_dimension = img_rows * img_cols
    model = CNN(input_dimension) # TODO add proper layers to CNN class above

    # Train
    train_model(train_batches, dev_batches, model)

    ## Evaluate the model on test data
    loss, acc = run_epoch(test_batches, model.eval(), None)
    print('Test loss1: {:.6f}  accuracy1: {:.6f}  loss2: {:.6f}   accuracy2: {:.6f}'.format(loss[0], acc[0], loss[1], acc[1]))

if __name__ == '__main__':
    # Specify seed for deterministic behavior, then shuffle. Do not change seed for official submissions to edx
    np.random.seed(12321)  # for reproducibility
    torch.manual_seed(12321)  # for reproducibility
    main()


C:\Users\ASUS\AppData\Local\Temp\ipykernel_11420\956033108.py:26: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  'y': torch.tensor([y_data[0][i:i + batch_size],


-------------
Epoch 1:


100%|██████████| 562/562 [00:31<00:00, 17.72it/s]


Train | loss1: 0.821761  accuracy1: 0.731067 | loss2: 0.868580  accuracy2: 0.705377


100%|██████████| 62/62 [00:01<00:00, 45.00it/s]


...

Train | loss1: 0.022990  accuracy1: 0.991937 | loss2: 0.029823  accuracy2: 0.989268


100%|██████████| 62/62 [00:01<00:00, 42.33it/s]


Valid | loss1: 0.075581  accuracy1: 0.983619 | loss2: 0.055958  accuracy2: 0.983115


100%|██████████| 62/62 [00:01<00:00, 42.70it/s]

Test loss1: 0.073952  accuracy1: 0.981351  loss2: 0.086194   accuracy2: 0.975050


In [16]:
class ModifiedCNN(nn.Module):
    def __init__(self, input_dimension):
        super(ModifiedCNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, (3, 3))
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, (3, 3))
        self.bn2 = nn.BatchNorm2d(64)

        self.flatten = Flatten()

        self.fc1 = nn.Linear(2880, 128)
        self.dropout1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)

        self.fc3 = nn.Linear(64, 20)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, (2, 2))

        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, (2, 2))

        x = self.flatten(x)

        x = F.relu(self.fc1(x))
        x = self.dropout1(x)

        x = F.relu(self.fc2(x))
        x = self.dropout2(x)

        x = self.fc3(x)

        out_first_digit = x[:,:10]
        out_second_digit = x[:,10:]

        return out_first_digit, out_second_digit

def main():
    # Load the data
    X_train, y_train, X_test, y_test = get_data(path_to_data_dir, use_mini_dataset)

    # Split into train and dev
    dev_split_index = int(9 * len(X_train) / 10)
    X_dev = X_train[dev_split_index:]
    y_dev = [y_train[0][dev_split_index:], y_train[1][dev_split_index:]]
    X_train = X_train[:dev_split_index]
    y_train = [y_train[0][:dev_split_index], y_train[1][:dev_split_index]]

    permutation = np.array([i for i in range(len(X_train))])
    np.random.shuffle(permutation)
    X_train = [X_train[i] for i in permutation]
    y_train = [[y_train[0][i] for i in permutation], [y_train[1][i] for i in permutation]]

    # Split dataset into batches
    train_batches = batchify_data(X_train, y_train, batch_size)
    dev_batches = batchify_data(X_dev, y_dev, batch_size)
    test_batches = batchify_data(X_test, y_test, batch_size)

    # Load the modified model
    model = ModifiedCNN(img_rows * img_cols)

    # Use Adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate if necessary

    # Train the model
    train_model(train_batches, dev_batches, model)

    # Evaluate the model on test data
    loss, acc = run_epoch(test_batches, model.eval(), None)
    print('Test loss1: {:.6f}  accuracy1: {:.6f}  loss2: {:.6f}   accuracy2: {:.6f}'.format(loss[0], acc[0], loss[1], acc[1]))


In [17]:
if __name__ == '__main__':
    main()


C:\Users\ASUS\AppData\Local\Temp\ipykernel_11420\956033108.py:26: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  'y': torch.tensor([y_data[0][i:i + batch_size],


-------------
Epoch 1:


100%|██████████| 562/562 [00:39<00:00, 14.08it/s]


Train | loss1: 1.241333  accuracy1: 0.564530 | loss2: 1.290240  accuracy2: 0.536421


100%|██████████| 62/62 [00:01<00:00, 37.53it/s]


...

Train | loss1: 0.119076  accuracy1: 0.958964 | loss2: 0.121931  accuracy2: 0.955655


100%|██████████| 62/62 [00:01<00:00, 34.03it/s]


Valid | loss1: 0.073040  accuracy1: 0.985383 | loss2: 0.065144  accuracy2: 0.985383


100%|██████████| 62/62 [00:01<00:00, 36.10it/s]


Test loss1: 0.095378  accuracy1: 0.982863  loss2: 0.093674   accuracy2: 0.981099
